In [2]:
import pandas as pd

### 1) joins, aggregates, aliases, where, group by, order by
Creates a query that lists each movie, the film category it is classified in, and the number of times it has been rented out.

``` mysql
SELECT f.title, c.name AS category, COUNT(r.rental_id) AS rental_count
FROM category c
JOIN film_category fc
ON c.category_id = fc.category_id
JOIN film f
ON f.film_id = fc.film_id
JOIN inventory i
ON f.film_id = i.film_id
JOIN rental r
ON i.inventory_id = r.inventory_id
WHERE c.name IN ('Animation', 'Children', 'Classics', 'Comedy', 'Family', 'Music')
GROUP BY 1, 2
ORDER BY 2, 1
```

### 2) window function, quartiles, IN
Provides a table with the movie titles and divide them into 4 levels (first_quarter, second_quarter, third_quarter, and final_quarter) based on the quartiles (25%, 50% 75%) of the rental duration for movies across all categories.

``` mysql
SELECT f.title,
       c.name AS category,
       f.rental_duration,
       NTILE(4) OVER(ORDER BY f.rental_duration) AS quartile
FROM category c
JOIN film_category fc
ON c.category_id = fc.category_id
JOIN film f
ON f.film_id = fc.film_id
WHERE c.name IN ('Animation', 'Children', 'Classics', 'Comedy', 'Family', 'Music')
```

### 3) common table expressions (CTEs)
Returns a table with the family-friendly film category, each of the quartiles, and the corresponding count of movies within each combination of film category for each corresponding rental duration category.

``` mysql
WITH cat_quart AS 
    (SELECT f.title, c.name AS category, f.rental_duration, 
            NTILE(4) OVER(ORDER BY f.rental_duration) AS quartile
     FROM category c
     JOIN film_category fc
     ON c.category_id = fc.category_id
     JOIN film f
     ON f.film_id = fc.film_id
     WHERE c.name IN ('Animation', 'Children', 'Classics', 
                       'Comedy', 'Family', 'Music'))

SELECT category, quartile, COUNT(*)
FROM cat_quart
GROUP BY 1, 2
ORDER BY 1, 2
```

### 4) sub-queries
Inquiry 1: How do the total number of rentals for each family-friendly genre compare to the average total number of rentals across all genres?

*Show each family friendly genre and the number of rental instances for each.*

``` mysql
SELECT category, SUM(rental_count) AS total_rentals
FROM (SELECT f.title, c.name AS category, COUNT(r.rental_id) AS rental_count
      FROM category c
      JOIN film_category fc
      ON c.category_id = fc.category_id
      JOIN film f
      ON f.film_id = fc.film_id
      JOIN inventory i
      ON f.film_id = i.film_id
      JOIN rental r
      ON i.inventory_id = r.inventory_id
      WHERE c.name IN ('Animation', 'Children', 'Classics', 'Comedy', 'Family', 'Music')
      GROUP BY 1, 2) fam_movies
GROUP BY 1
```

*Show average total rentals across all genres.*

``` mysql
WITH base AS (SELECT category, SUM(rental_count) AS total_rentals
              FROM (SELECT f.title, c.name AS category, COUNT(r.rental_id) AS rental_count
              FROM category c
              JOIN film_category fc
              ON c.category_id = fc.category_id
              JOIN film f
              ON f.film_id = fc.film_id
              JOIN inventory i
              ON f.film_id = i.film_id
              JOIN rental r
              ON i.inventory_id = r.inventory_id
              GROUP BY 1, 2) fam_movies
              GROUP BY 1)
SELECT AVG(total_rentals)
FROM base
```

### Inquiry 2
All payment dates are from 2007. How do the two stores’ running revenues compare over the 2007 year?